<a href="https://colab.research.google.com/github/omerfarukballi/StockPricePredictor/blob/master/Apple_Stock_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install numpy pandas sklearn keras yfinance matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached sklearn-0.0.post5.tar.gz (3.7 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [22]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go

# Download historical data as dataframe
ticker = "AAPL"  # Apple Inc.
df = yf.download(ticker, start="2010-01-01")

# Calculate the moving average
df['Moving_Avg'] = df['Close'].rolling(window=20).mean()

# Create a Plotly figure
fig = go.Figure()

# Add Close price trace
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['Close'],
    mode='lines',
    name='Close Price',
))

# Add moving average trace
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['Moving_Avg'],
    mode='lines',
    name='20 Day Moving Average',
))

# Set layout properties
fig.update_layout(
    title=f'{ticker} Stock Price and Moving Average',
    xaxis_title='Date',
    yaxis_title='Price ($)',
)

# Display the figure
fig.show()


[*********************100%***********************]  1 of 1 completed


In [19]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import yfinance as yf

# Download historical data as dataframe
ticker = "AAPL"  # Apple Inc.
df = yf.download(ticker, start="2010-01-01")

# Prepare data
data = df.filter(['Close'])
dataset = data.values
training_data_len = int(np.ceil( len(dataset) * .95 ))

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

# Create training dataset
train_data = scaled_data[0:int(training_data_len), :]
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# Build LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train model
model.fit(x_train, y_train, batch_size=1, epochs=1)

# Test data set
test_data = scaled_data[training_data_len - 59:, :]
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])

# Convert data to numpy array
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

real_data = scaled_data[-60:, :]
real_data = np.reshape(real_data, (1, real_data.shape[0], 1))

prediction_future = []

for _ in range(7):
    next_prediction = model.predict(real_data)[0, 0]  # Predict the next value
    prediction_future.append(next_prediction)  # Append the prediction to the prediction list
    real_data = np.roll(real_data, -1)  # Shift all data to the left
    real_data[0, -1, 0] = next_prediction  # Insert the new prediction at the end

prediction_future = np.array(prediction_future)
prediction_future = prediction_future.reshape(-1, 1)

# Inverse transform
prediction_future = scaler.inverse_transform(prediction_future)

print(f"Predicted prices for the next week are: {prediction_future.flatten()}")


[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 18ms/step
Predicted prices for the next week are: [174.31721 173.85492 172.85745 171.55498 170.08696 168.53596 166.95111]
